# Export for manufacturing

This notebook shows how to export a printable jaw model so you can print it and then, make clear aligners.

In [ ]:
import os
import glob
import base64
import time
import requests
import json
import trimesh
import urllib
import numpy as np

# Define call rules

Please modify the following code blocks based on the credentials you obtained from us.

In [ ]:
# Chohotech service request URL, sent with the API documentation.
base_url = "<service request URL>"

# Chohotech file service URL, sent with the API documentation.
file_server_url = "<service file server URL>"

# The authentication header must be passed in. Please keep the TOKEN confidential!!! If it is leaked, please contact us immediately to reset it. All tasks using this TOKEN will be charged to your account.
zh_token = "<your company's service Token, sent with the contact>" # All API calls must be authenticated with the token.

user_group = "APIClient" # User group, usually named APIClient.

# Your company's user_id, sent with the API documentation.
user_id = "<your company's user_id>"

# If you have received creds.json, it will be read directly below.
if os.path.exists('../../creds.json'):
    creds = json.load(open('../../creds.json', 'r'))
    base_url = creds['base_url']
    file_server_url = creds['file_server_url']
    zh_token = creds['zh_token']
    user_id = creds['user_id']
    print("loaded creds from creds.json")

In [ ]:
def upload_file(file_name):
    ext = file_name.split('.')[-1]
    data = open('../../data/' + file_name, 'rb').read()
    resp = requests.get(file_server_url + f"/scratch/{user_group}/{user_id}/upload_url?" +
                        f"postfix={ext}", # Must specify postfix, i.e., file extension
                        headers={"X-ZH-TOKEN": zh_token}) # Get signed upload URL
    resp.raise_for_status()

    upload_url = resp.text[1:-1] # Returns a single string JSON "string", can also use json.loads(resp.text)

    resp = requests.put(upload_url, data) # No auth header is needed for uploading to the cloud storage service

    resp.raise_for_status()
    path = "/".join(urllib.parse.urlparse(upload_url).path.lstrip("/").split("/")[3:])
    urn = f"urn:zhfile:o:s:{user_group}:{user_id}:{path}"
    return urn

def run_job_and_get_results(json_call, timeout_sec):
    headers = {
      "Content-Type": "application/json",
      "X-ZH-TOKEN": zh_token
    }

    url = base_url + '/run'

    response = requests.request("POST", url, headers=headers, data=json.dumps(json_call))
    response.raise_for_status()
    create_result = response.json()
    run_id = create_result['run_id']
    print("workflow id is", run_id)
    url = base_url + f"/run/{run_id}"

    start_time = time.time()
    while time.time()-start_time < timeout_sec:
        time.sleep(0.3)
        response = requests.request("GET", url, headers=headers)
        result = response.json()
        if result['completed'] or result['failed']:
            break

    if not result['completed']:
        if result['failed']:
            raise ValueError("API failed due to " + str(result['reason_public']))
        raise TimeoutError("API timeout")

    print("API finished in {}s".format(time.time()-start_time))
    url = base_url + f"/data/{run_id}"
    response = requests.request("GET", url, headers=headers)
    return response.json()

def retrieve_data(urn):
    return requests.get(file_server_url + f"/file/download?" + urllib.parse.urlencode({
                        "urn": urn}),
                        headers={"X-ZH-TOKEN": zh_token}).content

def retrieve_mesh(mesh_file_json):
    resp = requests.get(file_server_url + f"/file/download?" + urllib.parse.urlencode({
                        "urn": mesh_file_json['data']}),
                        headers={"X-ZH-TOKEN": zh_token})
    return trimesh.load(trimesh.util.wrap_as_stream(resp.content), file_type=mesh_file_json['type'])

## Prepare data

Prepare watertight teeth mesh and gum mesh

In [ ]:
json_call = {
  "spec_group": "mesh-processing",
  "spec_name": "oral-denoise-prod",
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id,
  "input_data": {
      "mesh": {"type":"drc", "data": upload_file("upper_jaw_scan.drc")},
      "jaw_type": "Upper"
  },
}
result_upper_jaw = run_job_and_get_results(json_call, 300)

In [ ]:
json_call = {
  "spec_group": "mesh-processing",
  "spec_name": "wf-gum-only-generation",
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id,
  "input_data": {
      "teeth_dict": result_upper_jaw['teeth_comp']
  },
}
result_upper_jaw_gum = run_job_and_get_results(json_call, 150)

In [ ]:
json_call = {
  "spec_group": "mesh-processing",
  "spec_name": "close-teeth-bottom",
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id,
  "input_data": {
      "teeth": result_upper_jaw['teeth_comp']
  },
}
result_close_bottom = run_job_and_get_results(json_call, 150)

## Generate printable jaw model

Please refer to https://www.chohotech.com/docs/cloud-en/#/module/dental-mesh-export-1

In [ ]:
json_call = {
  "spec_group": "mesh-processing",
  "spec_name": "dental-mesh-export",
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id,
  "input_data": {
    "gum": result_upper_jaw_gum['gum'],
    "tooth_dict": result_close_bottom['teeth'],
    "text": "Sample_11",
    "jaw_type": "Upper"
  },
  "output_config": {
      "mesh": {"type": "ply"}
  }
}
result_export= run_job_and_get_results(json_call, 300)

# Visualization

Following is the sample printable jaw model

In [ ]:
retrieve_mesh(result_export['result']['mesh']).show()